POINTS AND TARGETS

In [2]:
!pip install huggingface_hub sentence_transformers
!huggingface-cli login --token ''

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python -m pip install --upgrade pip
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import uuid

import pandas as pd
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
from datasets import load_dataset, Dataset, DatasetDict

train_dataset = load_dataset("Muhammad2003/routing-dataset", split="train")
train_dataset = train_dataset.rename_column("question", "sentence")

Generating train split:   0%|          | 0/14272 [00:00<?, ? examples/s]

In [5]:
train_dataset.set_format('pandas')
df = train_dataset[:14271]

df_3B = df[df['label'] == '3B']
df_7B = df[df['label'] == '7B']
df_30B = df[df['label'] == '30B']
df_70B = df[df['label'] == '70B']

print(df_3B.shape,df_7B.shape, df_30B.shape, df_70B.shape,)

(3568, 2) (3568, 2) (3568, 2) (3567, 2)


In [6]:
def _points():
    embedding_model = SentenceTransformer("Muhammad2003/router-embedding")
    df = pd.DataFrame(
        [
            *[
                {
                    "point_uid": str(uuid.uuid4()),
                    "category": "3B",
                    "utterance": utterance,
                }
                for utterance in df_3B[:]['sentence']
            ],
            *[
                {
                    "point_uid": str(uuid.uuid4()),
                    "category": "7B",
                    "utterance": utterance,
                }
                for utterance in df_7B[:]['sentence']
            ],
            *[
                {
                    "point_uid": str(uuid.uuid4()),
                    "category": "30B",
                    "utterance": utterance,
                }
                for utterance in df_30B[:]['sentence']
            ],
            *[
                {
                    "point_uid": str(uuid.uuid4()),
                    "category": "70B",
                    "utterance": utterance,
                }
                for utterance in df_70B[:]['sentence']
            ],
        ]
    )
    df["embedding"] = embedding_model.encode(df["utterance"].tolist()).tolist()
    return df

In [7]:
def _targets(points):
    return pd.DataFrame(
        [
            {
                "point_uid": point["point_uid"],
                "target": point["category"],
                "score": 1,
            }
            for _, point in points.iterrows()
        ]
    )

In [8]:
points = _points()
with open("points.jsonl", "w") as f:
    print(points.to_json(orient="records", lines=True), file=f)
targets = _targets(points)
with open("targets.jsonl", "w") as f:
    print(targets.to_json(orient="records", lines=True), file=f)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="./points.jsonl",
    path_in_repo="points.jsonl",
    repo_id="Muhammad2003/routing-dataset",
    repo_type="model",
)
api.upload_file(
    path_or_fileobj="./targets.jsonl",
    path_in_repo="targets.jsonl",
    repo_id="Muhammad2003/routing-dataset",
    repo_type="model",
)